<a href="https://colab.research.google.com/github/Darainer/Sunbot/blob/main/SunriseOnFalieChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summary of resources used:



In [4]:
# only needed if not running inside docker
#!pip install langchain
#!pip install unstructured
#!pip install faiss-cpu
#!pip install openai
#!pip install tiktoken
# %pip install python-dotenv

In [5]:
# mount my drive to hide API key -> this is a poor solution since it asks for authentication every time

#import os
#from google.colab import drive
#drive.mount('/content/drive/MyDrive/Colab_share')
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
#!ls "/content/drive/My Drive"
#!source /content/drive/MyDrive/Colab_share/env_vars.sh

In [6]:
# scrape all the pages, for now manually downsampling the raw_url to a smaller subset
import requests
from bs4 import BeautifulSoup

url = 'https://www.sunriseonfalie.com.au/'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

raw_urls = []
for link in soup.find_all('a'):
	raw_urls.append(link.get('href'))

raw_urls

['https://www.facebook.com/sunriseonfalie',
 'https://www.instagram.com/sunriseonfalie/',
 'https://www.sunriseonfalie.com.au/about-us/eco-story/',
 'tel:0417 897 715',
 'home/',
 'contact-us-and-bookings/',
 'our-apartments/',
 'https://book-directonline.com/properties/sunriseonfaliedirect',
 'javascript:void(0)',
 'about-us/guest-reviews/',
 'about-us/eco-story/',
 'about-us/well-behaved-pets-are-welcome/',
 'about-us/your-hosts/',
 'about-us/our-story/',
 'about-us/getting-to-sunrise-on-falie/',
 'about-us/super-hosts-super-clean/',
 'about-us/sunrise-photography/',
 'useful-kangaroo-island-info',
 'disability-friendly-accommodation',
 'javascript:void(0)',
 'kangaroo-island-itineraries/linger-longer/',
 'kangaroo-island-itineraries/best-time-to-visit-ki/',
 'kangaroo-island-photo-gallery',
 'javascript:void(0)',
 'kangaroo-island-attractions/experiences/',
 'kangaroo-island-attractions/nature/',
 'kangaroo-island-attractions/local-dining/',
 'kangaroo-island-attractions/local-produ

In [7]:
urls= [
 'https://www.sunriseonfalie.com.au/home/',
 'https://www.sunriseonfalie.com.au/contact-us-and-bookings/',
 'https://www.sunriseonfalie.com.au/our-apartments/',
 'https://www.sunriseonfalie.com.au/about-us/guest-reviews/',
 'https://www.sunriseonfalie.com.au/about-us/your-hosts/',
 'https://www.sunriseonfalie.com.au/about-us/our-story/',
 'https://www.sunriseonfalie.com.au/about-us/well-behaved-pets-are-welcome/',
 'https://www.sunriseonfalie.com.au/about-us/eco-story/',
 'https://www.sunriseonfalie.com.au/about-us/getting-to-sunrise-on-falie/',
 'https://www.sunriseonfalie.com.au/about-us/super-hosts-super-clean/',
 'https://www.sunriseonfalie.com.au/about-us/sunrise-photography/',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info',
 'https://www.sunriseonfalie.com.au/disability-friendly-accommodation',
 'https://www.sunriseonfalie.com.au/kangaroo-island-itineraries/linger-longer/',
 'https://www.sunriseonfalie.com.au/kangaroo-island-itineraries/best-time-to-visit-ki/',
 'https://www.sunriseonfalie.com.au/kangaroo-island-itineraries/linger-longer/2-day-1-night-quick-visit',
 'https://www.sunriseonfalie.com.au/kangaroo-island-itineraries/linger-longer/4-day-3-night-itinerary',
 'https://www.sunriseonfalie.com.au/kangaroo-island-itineraries/linger-longer/5-day-4-night-exploration',
 'https://www.sunriseonfalie.com.au/kangaroo-island-photo-gallery',
 'https://www.sunriseonfalie.com.au/kangaroo-island-attractions/experiences/',
 'https://www.sunriseonfalie.com.au/kangaroo-island-attractions/nature/',
 'https://www.sunriseonfalie.com.au/kangaroo-island-attractions/local-dining/',
 'https://www.sunriseonfalie.com.au/kangaroo-island-attractions/local-products/',
 'https://www.sunriseonfalie.com.au/kangaroo-island-attractions/local-artists-and-galleries/',
 'https://www.sunriseonfalie.com.au/our-apartments/apartment-1-horizon',
 'https://www.sunriseonfalie.com.au/our-apartments/apartment-2-deluxe',
 'https://www.sunriseonfalie.com.au/our-apartments/apartment-3-penthouse-retreat',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/fishing',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/eats',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/township',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/wildlife',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/walks',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/art',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/fishing',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/roads',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/seasons',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/locals',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/cycling',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/history',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/regrowth',
 'https://www.sunriseonfalie.com.au/useful-kangaroo-island-info/kelly-hill-caves'
]

In [8]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

from langchain.document_loaders import UnstructuredURLLoader
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()

[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/vscode/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
#data

In [10]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n',
                                      chunk_size=1000,
                                      chunk_overlap=200)


docs = text_splitter.split_documents(data)

Created a chunk of size 1119, which is longer than the specified 1000
Created a chunk of size 1119, which is longer than the specified 1000


In [11]:
#docs

In [12]:
len(docs)

256

Managing the OPENAI_API_KEY securely.  
- Local use with .env file  
- Codespace / Github actions use the "secrets" functionality to set the env variable.

In [13]:
from dotenv import load_dotenv
import os

# Check if the key is already in the environment (codespace and action secrets)
if not os.environ.get("OPENAI_API_KEY"):
    # Try loading from .env file if on local dev environment
    if not load_dotenv():
        raise ValueError("OPENAI_API_KEY not found")

# define your own key here as an environment variable, if you are copying this. 
# os.environ["OPENAI_API_KEY"] = "Your_key_here********"

In [14]:
import pickle
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [15]:
vectorStore_openAI = FAISS.from_documents(docs, embeddings)

with open("faiss_store_openai.pkl", "wb") as f:
       pickle.dump(vectorStore_openAI, f)

In [16]:
with open("faiss_store_openai.pkl", "rb") as f:
    VectorStore = pickle.load(f)

In [17]:
VectorStore

In [18]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI
from langchain.memory import ConversationBufferMemory


In [19]:
llm=OpenAI(temperature=0, model_name='text-davinci-003')

In [20]:
# llm

In [21]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VectorStore.as_retriever())

The above sections create the model and give it the context of the website. The following section is some questions to test the understanding of the model

We want to remember the conversation up to the context limit of the model

In [22]:
memory = ConversationBufferMemory()  # todo: get context limit of model

In [23]:
chain({"question": "Which apartment at sunrise on Falie has the best view?"}, return_only_outputs=True)


{'answer': ' It is not possible to determine which apartment at Sunrise on Falie has the best view.\n',
 'sources': 'https://www.sunriseonfalie.com.au/about-us/our-story/, https://www.sunriseonfalie.com.au/about-us/guest-reviews/, https://www.sunriseonfalie.com.au/kangaroo-island-photo-gallery'}

In [24]:
chain({"question": "What are the best eating locations in american river"}, return_only_outputs=True)

{'answer': ' The best eating locations in American River are The Deck Café and local charter boats.\n',
 'sources': ''}

In [25]:
chain({"question": "Can i swim in the water on kangaroo island"}, return_only_outputs=True)

{'answer': ' Yes, you can swim in the water on Kangaroo Island at Stokes Bay, Island Beach and Emu Bay.\n',
 'sources': 'https://www.sunriseonfalie.com.au/home/, https://www.sunriseonfalie.com.au/kangaroo-island-itineraries/linger-longer/5-day-4-night-exploration'}

In [26]:
chain({"question": "when is the best time to visit kangaroo island"}, return_only_outputs=True)

{'answer': " The best time to visit Kangaroo Island is July. It is not too cold (8-15C 46-59F) and it is when you might spot newborn roos and wallabies emerging from their mother's prouches.\n",
 'sources': ''}

In [27]:
chain({"question": "i have a wheelchair, can i stay in this accomodation?"}, return_only_outputs=True)

{'answer': ' Yes, guests in wheelchairs can stay in this accommodation. The apartments are designed to be highly accessible for people in wheelchairs, with features such as ramp access, disability accessible bathrooms, wide halls and doorways, wide outdoor decks, floor to ceiling windows, and wheelchair-equipped bathrooms and kitchens.\n',
 'sources': ''}

In [28]:
chain({"question": "i have a pet dog, can i bring it to stay with me?"}, return_only_outputs=True)

{'answer': ' Yes, you can bring your pet dog to stay with you at Sunrise on Falie, as long as it is well behaved and house trained, and you arrange the additional fee of $30 with the owner prior.\n',
 'sources': ''}

In [29]:
chain({"question": "i have only 2 days, can you suggest an itenary?"}, return_only_outputs=True)

{'answer': ' Please refer to our shorter itineraries for suggestions, such as the 2 Day - 1 Night quick visit itinerary which suggests arriving by ferry or plane and exploring the island to maximize daylight sightseeing.\n',
 'sources': ''}

In [30]:
chain({"question": "what are the contact details for Sunrise on Falie?"}, return_only_outputs=True)

{'answer': ' The contact details for Sunrise on Falie are to send an email.\n',
 'sources': ''}

In [31]:
# todo: check if doc search is happening on the above queries. alternate format
#query = "who are the authors of the article?"
#docs = docsearch.similarity_search(query)
#chain.run(input_documents=docs, question=query)

In [32]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("whats up?")